In [1]:
import numpy as np
import pandas as pd # for loading data
import sklearn
import matplotlib.pyplot as plt
import re # for removing special characters
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
import nltk
import random
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords # For removing stopwords
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

%matplotlib inline


[nltk_data] Downloading package wordnet to /home/paul/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/paul/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
cd ..

/mnt/c/Users/paulp/Desktop/machine learning


In [3]:
from util.helperfunctions import clean_text

/home/paul/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/paul/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/paul/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/paul/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWa

In [4]:
cd src

/mnt/c/Users/paulp/Desktop/machine learning/src


In [5]:
validation_df = pd.read_csv("validation.csv")
lyrics_val, dummy_list = clean_text(validation_df['Lyrics'], 0)
genre_val = validation_df['Genre']

In [6]:
train_df = pd.read_csv("train.csv")
lyrics_train, dummy_list= clean_text(train_df['Lyrics'], 0)
genre_train = train_df['Genre']

In [7]:
test_df = pd.read_csv("test.csv")
lyrics_test, dummy_list = clean_text(test_df['Lyrics'], 0)
genre_test = test_df['Genre']

using a count vectorizer gave us better results than actually using the TFIDF formula we hypothesize that normalizing the frequently used terms is actually not desirable with so many classes

In [8]:
vectorizer = CountVectorizer(min_df = 3)
train_data_tfid = vectorizer.fit_transform(lyrics_train).toarray()
test_data = vectorizer.transform(lyrics_test).toarray()
validation_data = vectorizer.transform(lyrics_val).toarray()

In [9]:
# Classification using LogisticRegression
not_done = True
acc_old = 0
acc_new = 0
alpha = 0.
alpha_rate_small = 1.
alpha_rate_big = 3.
while(not_done):
    naive_bayes = MultinomialNB(alpha)
    naive_bayes.fit(train_data_tfid, genre_train)
    predictions_val = naive_bayes.predict(validation_data)
    acc_new = accuracy_score(genre_val, predictions_val)
    if acc_old > acc_new:
        alpha = alpha + alpha_rate_big
        naive_bayes = MultinomialNB(alpha)
        naive_bayes.fit(train_data_tfid, genre_train)
        predictions_val = naive_bayes.predict(validation_data)
        acc_new = accuracy_score(genre_val, predictions_val)
        if acc_old > acc_new:
            alpha = alpha - alpha_rate_big
            naive_bayes = MultinomialNB(alpha)
            naive_bayes.fit(train_data_tfid, genre_train)
            not_done = False
            break
    acc_old = acc_new
    print('Accuracy score naive_bayes in training: {} and alpha: {}'.format(acc_old, alpha))
    alpha = alpha + alpha_rate_small
        
predictions1 = naive_bayes.predict(test_data)
print('final Accuracy score naive_bayes on test data = {}'.format(accuracy_score(genre_test, predictions1)))
predictions1 = naive_bayes.predict(train_data_tfid)
print('final Accuracy score naive_bayes on training data = {}'.format(accuracy_score(genre_train, predictions1)))

not_done = True
acc_old = 0
acc_new = 0
alpha = 0.
alpha_rate_small = 1.
alpha_rate_big = 3.
while(not_done):
    gnb_clf = GaussianNB(priors=None, var_smoothing=alpha)
    gnb_clf.fit(train_data_tfid, genre_train)
    predictions_val = gnb_clf.predict(validation_data)
    acc_new = accuracy_score(genre_val, predictions_val)
    if acc_old > acc_new:
        alpha = alpha + alpha_rate_big
        gnb_clf = GaussianNB(priors=None, var_smoothing=alpha)
        gnb_clf.fit(train_data_tfid, genre_train)
        predictions_val = gnb_clf.predict(validation_data)
        acc_new = accuracy_score(genre_val, predictions_val)
        if acc_old > acc_new:
            alpha = alpha - alpha_rate_big
            gnb_clf = GaussianNB(priors=None, var_smoothing=alpha)
            gnb_clf.fit(train_data_tfid, genre_train)
            not_done = False
            break
    acc_old = acc_new
    print('Accuracy score naive_bayes gaussian in training: {} and alpha: {}'.format(acc_old, alpha))
    alpha = alpha + alpha_rate_small

predictions2 = gnb_clf.predict(test_data)
print('Accuracy score gnb_clf on the test data = {}'.format(accuracy_score(genre_test, predictions2)))
predictions2 = gnb_clf.predict(train_data_tfid)
print('final Accuracy score gnb_clf on training data = {}'.format(accuracy_score(genre_train, predictions2)))

/usr/lib/python3/dist-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


Accuracy score naive_bayes in training: 0.4631185807656396 and alpha: 0.0
Accuracy score naive_bayes in training: 0.5032679738562091 and alpha: 1.0
final Accuracy score naive_bayes on test data = 0.4962962962962963
final Accuracy score naive_bayes on training data = 0.6742919389978214


TypeError: __init__() got an unexpected keyword argument 'var_smoothing'